In [1]:
#FARE
from numpy import *
import numpy as np
from scipy import *
import scipy
from scipy.sparse import *
from scipy.spatial.distance import *
from sklearn.metrics.pairwise import *

In [43]:
a=array([0,0,0,1,1,0,1,1,1,1,1])
b=array([1,1,1,1,1,0,0,0,0,0,1])
cosine(a,b)

0.53708995011372429

In [48]:
type(a)

numpy.ndarray

In [2]:
tracks_with_tags_ordered=load_npz("tracks_with_tags_bincount_ordered.npz")
tracks_with_tags_ordered

<100000x276615 sparse matrix of type '<class 'numpy.float64'>'
	with 487328 stored elements in Compressed Sparse Column format>

In [3]:
tracks_with_tags_ordered.getcol(0)

<100000x1 sparse matrix of type '<class 'numpy.float64'>'
	with 3846 stored elements in Compressed Sparse Column format>

In [3]:
IDtracksOrdered=load("ordered_tracks.npy")
IDtracksOrdered

array([ 1563309.,  1363985.,  3705881., ...,  2005630.,  4432895.,
        3626362.])

In [10]:
tracks_with_tags_ordered[1].nonzero()[1]

array([  3982,  11056, 127137, 170251, 191177], dtype=int32)

In [2]:
#FARE
def check_matrix(X, format='csc', dtype=np.float32):
    if format == 'csc' and not isinstance(X, csc_matrix):
        return X.tocsc().astype(dtype)
    elif format == 'csr' and not isinstance(X, csr_matrix):
        return X.tocsr().astype(dtype)
    elif format == 'coo' and not isinstance(X, coo_matrix):
        return X.tocoo().astype(dtype)
    elif format == 'dok' and not isinstance(X, dok_matrix):
        return X.todok().astype(dtype)
    elif format == 'bsr' and not isinstance(X, bsr_matrix):
        return X.tobsr().astype(dtype)
    elif format == 'dia' and not isinstance(X, dia_matrix):
        return X.todia().astype(dtype)
    elif format == 'lil' and not isinstance(X, lil_matrix):
        return X.tolil().astype(dtype)
    else:
        return X.astype(dtype)

In [4]:
#FARE
shrinkage=2

In [5]:
#FARE
def apply_shrinkage(X, dist):
        # create an "indicator" version of X (i.e. replace values in X with ones)
        X_ind = X.copy()
        X_ind.data = np.ones_like(X_ind.data)
        # compute the co-rated counts
        co_counts = X_ind * X_ind.T
        # remove the diagonal
        co_counts = co_counts - dia_matrix((co_counts.diagonal()[scipy.newaxis, :], [0]), shape=co_counts.shape)
        # compute the shrinkage factor as co_counts_ij / (co_counts_ij + shrinkage)
        # then multiply dist with it
        co_counts_shrink = co_counts.copy()
        co_counts_shrink.data += shrinkage
        co_counts.data /= co_counts_shrink.data
        dist.data *= co_counts.data
        return dist

In [6]:
#FARE
def myCos(X):
    X=check_matrix(X, 'csc', dtype=float32)
    Xsq=X.copy()
    Xsq.data **= 2
    norm= sqrt(Xsq.sum(axis=0))
    norm=asarray(norm).ravel()
    norm+= 1e-6
    col_nnz=diff(X.indptr)
    X.data /= repeat(norm,col_nnz)
    print("normalized")
    
    dist=X*X.T
    print("Computed")
    
    # zero out diagonal values
    dist = dist - dia_matrix((dist.diagonal()[scipy.newaxis, :], [0]), shape=dist.shape)
    print("Removed diagonal")
    
    if shrinkage > 0:
       dist = apply_shrinkage(X, dist)
       print("Applied shrinkage")    
        
    return dist

In [4]:
type(tracks_with_tags_ordered[0])

scipy.sparse.csc.csc_matrix

In [39]:
#FARE
playlist_with_tags=load_npz("playlists_with_tags.npz")
playlist_with_tags

<10000x276615 sparse matrix of type '<class 'numpy.float64'>'
	with 559049 stored elements in Compressed Sparse Column format>

In [48]:
normalize(playlist_with_tags)

<10000x276615 sparse matrix of type '<class 'numpy.float64'>'
	with 559049 stored elements in Compressed Sparse Row format>

In [52]:
playlist_with_tags[0].data

array([  1.,   1.,   2.,   1.,   1.,   1.,  22.,   1.,  20.,   5.,   1.,
         2.,   1.,  20.,   5.,  52.,   1.,   1.,   1.,   1.,   1.,   1.,
         1.,   1.,   1.,   6.,   1.,   1.,   1.,   1.,  53.,   6.,   1.,
        12.,   1.,   1.,   1.,   1.,   2.,   1.,   1.,   2.,   1.,  10.,
         1.,   1.,   1.,   1.,   1.,  35.,   2.,   1.,   1.,   1.,   1.,
         1.,   1.,   1.,   1.,   1.,   1.,   2.,   1.,   1.,   1.,   2.])

In [53]:
normalize(playlist_with_tags)[0].data

array([ 0.01087021,  0.01087021,  0.02174041,  0.01087021,  0.01087021,
        0.01087021,  0.23914456,  0.01087021,  0.21740415,  0.05435104,
        0.01087021,  0.02174041,  0.01087021,  0.21740415,  0.05435104,
        0.56525078,  0.01087021,  0.01087021,  0.01087021,  0.01087021,
        0.01087021,  0.01087021,  0.01087021,  0.01087021,  0.01087021,
        0.06522124,  0.01087021,  0.01087021,  0.01087021,  0.01087021,
        0.57612099,  0.06522124,  0.01087021,  0.13044249,  0.01087021,
        0.01087021,  0.01087021,  0.01087021,  0.02174041,  0.01087021,
        0.01087021,  0.02174041,  0.01087021,  0.10870207,  0.01087021,
        0.01087021,  0.01087021,  0.01087021,  0.01087021,  0.38045726,
        0.02174041,  0.01087021,  0.01087021,  0.01087021,  0.01087021,
        0.01087021,  0.01087021,  0.01087021,  0.01087021,  0.01087021,
        0.01087021,  0.02174041,  0.01087021,  0.01087021,  0.01087021,
        0.02174041])

In [41]:
#FARE
PLsimilarity=myCos(normalize(playlist_with_tags))

normalized
Computed
Removed diagonal
Applied shrinkage


In [55]:
argmax(PLsimilarity[0])

1718

In [56]:
unNormalized=myCos(playlist_with_tags)

normalized
Computed
Removed diagonal
Applied shrinkage


In [66]:
diff=0
for i in range(10000):
    if (argmax(PLsimilarity[i])!=argmax(unNormalized[i])):
        diff+=1
diff

KeyboardInterrupt: 

In [43]:
(PLsimilarity[0]).getcol(1718).data

array([ 1.62901485], dtype=float32)

In [29]:
type(PLsimilarity)

scipy.sparse.csc.csc_matrix

In [8]:
save_npz("playlist_similarity.npz",PLsimilarity)

In [37]:
max(PLsimilarity.data)

10.699376

In [30]:
testM=csc_matrix([[0,0,0,0,200,10],[20,0,0,0,100,0],[15,0,1,0,0,0]])

In [46]:
myCos(testM).todense()

normalized
Computed
Removed diagonal
Applied shrinkage


matrix([[ 0.        ,  0.13333333,  0.        ],
        [ 0.13333333,  0.        ,  0.15999998],
        [ 0.        ,  0.15999998,  0.        ]], dtype=float32)

In [47]:
myCos(normalize(testM)).todense()

normalized
Computed
Removed diagonal
Applied shrinkage


matrix([[ 0.        ,  0.16663836,  0.        ],
        [ 0.16663836,  0.        ,  0.06308009],
        [ 0.        ,  0.06308009,  0.        ]], dtype=float32)

In [3]:
N=5
K=18
T=30
startingArr=array([0]*(K-N)+[1]*N)
np.random.shuffle(startingArr)
for i in range(T-1):
    myArr=array([0]*(K-N)+[1]*N)
    np.random.shuffle(myArr)
    startingArr=np.vstack((startingArr,myArr))

In [4]:
startingArr

array([[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1],
       [0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1],
       [1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1],
       [0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0],
       [1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,

In [147]:
myCos(csc_matrix(startingArr))

normalized
Computed
Removed diagonal
Applied shrinkage


<30x30 sparse matrix of type '<class 'numpy.float32'>'
	with 746 stored elements in Compressed Sparse Column format>

In [277]:
type(startingArr)

numpy.ndarray

In [276]:
resizedArray=pad(startingArr,((0,32-30),(0,32-18)), "constant", constant_values=0)
coo_matrix(resizedArray)

<32x32 sparse matrix of type '<class 'numpy.int64'>'
	with 150 stored elements in COOrdinate format>

In [253]:
csc_matrix(startingArr)

<30x18 sparse matrix of type '<class 'numpy.int64'>'
	with 150 stored elements in Compressed Sparse Column format>

In [256]:
block1=csc_matrix(startingArr[0:14,0:9])
block2=csc_matrix(startingArr[0:14,9:18])
block3=csc_matrix(startingArr[14:30,0:9])
block4=csc_matrix(startingArr[14:30,9:18])
bmat([[block1,block2],[block3,block4]])

<30x18 sparse matrix of type '<class 'numpy.int64'>'
	with 150 stored elements in COOrdinate format>

In [269]:
rblock1=csc_matrix(resizedArray[0:16,0:16])
rblock2=csc_matrix(resizedArray[0:16,16:32])
rblock3=csc_matrix(resizedArray[16:32,0:16])
rblock4=csc_matrix(resizedArray[16:32,16:32])
bmat([[rblock1,rblock2],[rblock3,rblock4]])

<32x32 sparse matrix of type '<class 'numpy.int64'>'
	with 150 stored elements in COOrdinate format>

In [204]:
coo_matrix(startingArr)

<30x18 sparse matrix of type '<class 'numpy.int64'>'
	with 150 stored elements in COOrdinate format>

In [252]:
realProd=csc_matrix(resizedArray)*csc_matrix(resizedArray).T
coo_matrix(realProd)

<32x32 sparse matrix of type '<class 'numpy.int64'>'
	with 776 stored elements in COOrdinate format>

In [271]:
pb1=rblock1*rblock1.T+rblock2*rblock2.T
pb2=rblock1*rblock3.T+rblock2*rblock4.T
pb3=rblock3*rblock1.T+rblock4*rblock2.T
pb4=rblock3*rblock3.T+rblock4*rblock4.T

bmat([[pb1,pb2],[pb3,pb4]])

<32x32 sparse matrix of type '<class 'numpy.int64'>'
	with 776 stored elements in COOrdinate format>

In [274]:
tracks_with_tags_ordered.shape

(100000, 276615)

In [6]:
resizedTracks=pad(startingArr,((0,2**19-100000),(0,2**19-276615)), "constant", constant_values=0)

MemoryError: 

In [13]:
tracks_with_tags_ordered.dot(tracks_with_tags_ordered.T)

MemoryError: 

In [5]:
cscTracks=csc_matrix(tracks_with_tags_ordered)

In [1]:
cscTracks*cscTracks.T#memory error

NameError: name 'cscTracks' is not defined

In [6]:
cscTracks.shape

(100000, 276615)

In [7]:
cscTracks.size-cscTracks.getcol(0).size

483482

In [8]:
startingCol=cscTracks.getcol(22)
for i in range(23,276615):#0,22,26,27,28,44
    #si salta la 0 perchè non sono significativi
    thisCol=cscTracks.getcol(i)
    if thisCol.size!=0:
        startingCol=hstack((startingCol,thisCol))

In [9]:
startingCol

<100000x31900 sparse matrix of type '<class 'numpy.float64'>'
	with 483482 stored elements in Compressed Sparse Column format>

In [10]:
cscReduced=startingCol.astype(int8)

In [4]:
cscReduced

<100000x31900 sparse matrix of type '<class 'numpy.int8'>'
	with 483482 stored elements in Compressed Sparse Column format>

In [13]:
save_npz("track_tags_reduced.npz",cscReduced)

In [2]:
cscReduced=load_npz("track_tags_reduced.npz")

In [5]:
csrReduced=csr_matrix(cscReduced)

In [ ]:
cscReduced*cscReduced.T

In [ ]:
csrReduced*csrReduced.T

In [13]:
cscReduced.getrow(0).data

array([1, 1, 1, 1, 1], dtype=int8)